<H1>Create a simple cantilever line beam model</H1>


In [ ]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0")

In [ ]:
# Input Params
length = 5.0 #m		     Length of beam
point_load = 10.0  #kN   Concentrated load on cantilever tip
model_name = "Cantilever Line Beam" 

<div class="alert alert-block alert-info">
<b>Tip:</b> When interacting with Modeller externally, as we are here. A large speed up can be achived by disabling the user interface. In v22 and later this can be done by achieved with the enableUI function in earlier versions the application must be set to be invisible. Take care to restore the visibility and watch out for exceptions in the code that can leave Modeller running in the background or disabled. See the code below to ensure this is properly restored.
</div>

In [ ]:
do_speed_up = False # To speed up interaction with Modeller from python set this to True

if do_speed_up: 
    if lusas.getMajorVersionNumber() >= 22:
        lusas.enableUI(False)
    else:
        lusas.setVisible(False)

In [ ]:
# Create a new model
db = lusas.newDatabase("Structural", f"{model_name}.mdl")
# Set the vertical axis
db.setLogicalUpAxis("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [ ]:
''' Create a line'''
# Get the geometry data object
geomData = lusas.geometryData()
# Set the defaults
geomData.setAllDefaults()
# Set the coordinates of the first point
geomData.addCoords(0, 0, 0)
# Set the coordinates of the second point
geomData.addCoords(length, 0, 0)
# Create the line object from the geomety data
objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
# Get the lines
lines = objs.getObjects("Lines")
# Get the single line from the returned array
beamLine = lines[0]

In [ ]:
''' Create a mesh attribute'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
meshAttr.setSize("BMI21", 1)

In [ ]:
''' Create a geometric attribute'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the element type
geomAttr.setValue("elementType", "3D Thick Beam")
# Set the beam section properties for the section library
geomAttr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "406x178x74kg UB", 0, 0, 0)

In [ ]:
''' Create a material attribute'''
materialAttr = db.createIsotropicMaterial("Steel", 209000000.0, 0.3, 7.8)

In [ ]:
''' Create a support attribute (fixed) '''
fixedSupport = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

In [ ]:
''' Create a point load attribute '''
loadAttr = db.createLoadingConcentrated("Point load")
loadAttr.setConcentrated(0.0, 0.0, -point_load)

In [ ]:
''' Assign the attributes to the geometry '''
# get the assignment object
assignment = lusas.assignment()
# set the defaults
assignment.setAllDefaults()

# Assign the mesh
meshAttr.assignTo(beamLine, assignment)
# Assign the geometry
geomAttr.assignTo(beamLine, assignment)
# Assign the material
materialAttr.assignTo(beamLine, assignment)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(beamLine.getStartPoint(), assignment)


<div class="alert alert-block alert-warning">
<b>Note:</b> If the code fails before this point the cell below must be run manually to restore the user interface
</div>

In [ ]:
if lusas.getMajorVersionNumber() >= 22:
    lusas.enableUI(True)
else:
    lusas.setVisible(True)